In [1]:
import data_generator 
from sample_models import *
import numpy as np
from data_generator import AudioGenerator
from keras import backend as K
from utils import int_sequence_to_text

from keras.layers import (BatchNormalization, Conv1D, Dense, Input, 
    TimeDistributed, Activation, Bidirectional, SimpleRNN, GRU, LSTM,Dropout)

Using TensorFlow backend.


In [ ]:
def TDNN_LSTM(input_dim, output_dim=29):
    """ Build a deep network for speech 
    """  
    # Main acoustic input
    input_data = Input(name='the_input', shape=(None, input_dim))
    
    time_dense1 = TimeDistributed(Dense(200))(input_data)
    time_dense2 = TimeDistributed(Dense(200))(time_dense1)
    time_dense3 = TimeDistributed(Dense(200))(time_dense2)

    # Add batch normalization
    bn_td1 = BatchNormalization(name='bn_td1')(time_dense3)
    lstm1 = LSTM(200, activation='tanh',return_sequences=True)(bn_td1)
    
    time_dense4 = TimeDistributed(Dense(150))(lstm1)
    time_dense5 = TimeDistributed(Dense(150))(time_dense4)

    # Add batch normalization
    bn_td2 = BatchNormalization(name='bn_td2')(time_dense5)    
    lstm2 = LSTM(150, activation='tanh',return_sequences=True)(bn_td2)
    
    time_dense6 = TimeDistributed(Dense(100))(lstm2)
    time_dense7 = TimeDistributed(Dense(100))(time_dense6)

    # Add batch normalization
    bn_td3 = BatchNormalization(name='bn_td2')(time_dense7)    
    lstm3 = LSTM(80, activation='tanh',return_sequences=True)(bn_td3)
    
    time_dense = TimeDistributed(Dense(output_dim))(lstm2)
    # TODO: Add softmax activation layer
    y_pred = Activation('softmax', name='softmax')(time_dense)
    # Specify the model
    model = Model(inputs=input_data, outputs=y_pred)
    # TODO: Specify model.output_length
    model.output_length = lambda x: x
    print(model.summary())
    return model

In [ ]:
model = TDNN_LSTM(161, output_dim=29)

In [ ]:
from train_utils import train_model

train_model(input_to_softmax=model, 
            pickle_path='model_lstm.pickle', 
            save_model_path='model_lstm.h5', 
            spectrogram=True)

In [11]:
def cnn_rnn_3_model(input_dim, filters, kernel_size, conv_stride,
    conv_border_mode, units, output_dim=29):
    """ Build a recurrent + convolutional network for speech 
    """
    # Main acoustic input
    input_data = Input(name='the_input', shape=(None, input_dim))
    # Add convolutional layer
    conv_1d = Conv1D(filters, kernel_size, 
                     strides=conv_stride, 
                     padding=conv_border_mode,
                     activation='relu',
                     name='conv1d')(input_data)
    drop1 = Dropout(0.1)(conv_1d)
    # Add batch normalization
    
    # Add batch normalization
    bn_cnn2 = BatchNormalization(name='bn_conv_1d2')(drop1)
    
    # Add a recurrent layer
    simp_rnn = SimpleRNN(units, activation='relu',
        return_sequences=True, implementation=2, name='rnn')(bn_cnn2)
    # TODO: Add batch normalization
    bn_rnn = BatchNormalization(name='bn_rnn')(simp_rnn)
    
    
    # TODO: Add a TimeDistributed(Dense(output_dim)) layer
    time_dense = TimeDistributed(Dense(output_dim))(bn_rnn)
    
    
    # Add softmax activation layer
    y_pred = Activation('softmax', name='softmax')(time_dense)
    # Specify the model
    model = Model(inputs=input_data, outputs=y_pred)
    model.output_length = lambda x: cnn_output_length(
        x, kernel_size, conv_border_mode, conv_stride)
    print(model.summary())
    return model

In [12]:
model_2 = cnn_rnn_3_model(input_dim=161, # change to 13 if you would like to use MFCC features
                        filters=200,
                        kernel_size=11, 
                        conv_stride=2,
                        conv_border_mode='valid',
                        units=200)

//anaconda/envs/tensorflow/lib/python3.5/site-packages/keras/layers/recurrent.py:1028: UserWarning: The `implementation` argument in `SimpleRNN` has been deprecated. Please remove it from your layer call.
  warnings.warn('The `implementation` argument '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       (None, None, 161)         0         
_________________________________________________________________
conv1d (Conv1D)              (None, None, 200)         354400    
_________________________________________________________________
dropout_5 (Dropout)          (None, None, 200)         0         
_________________________________________________________________
bn_conv_1d2 (BatchNormalizat (None, None, 200)         800       
_________________________________________________________________
rnn (SimpleRNN)              (None, None, 200)         80200     
_________________________________________________________________
bn_rnn (BatchNormalization)  (None, None, 200)         800       
_________________________________________________________________
time_distributed_4 (TimeDist (None, None, 29)          5829      
__________

In [13]:
from train_utils import train_model

train_model(input_to_softmax=model_2, 
            pickle_path='model_3.pickle', 
            save_model_path='model_3.h5', 
            spectrogram=True)

Epoch 1/20
  3/101 [..............................] - ETA: 5:06 - loss: 597.8899

KeyboardInterrupt: 